<a href="https://colab.research.google.com/github/dohyeon-kim012/WebScraping/blob/main/4.HTML%20%EC%8A%A4%ED%81%AC%EB%A0%88%EC%9D%B4%ED%95%91_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 무신사 스크레이핑

In [ ]:
from urllib.request import urlretrieve # 이미지 파일 저장 용도
import requests # url 접속 용도

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
url = 'https://search.musinsa.com/category/001001'
data = requests.get(url)
data

<Response [200]>

In [ ]:
musinsa_main = data.content
soup = BeautifulSoup(musinsa_main, 'html.parser')

In [ ]:
soup

**여기서 전략을 잘 짜야 함!**  

1. **page** 파라미터  

## **pagenation** 
: page 단위로 정보를 크롤링하기 때문에, page당 정보가 어떻게 변경되는지가 중요

페이지네이션 크롤링 시 참고할 수 있는 기법
1. 몇 페이지가 마지막 페이지 인지를 알 수 있는 경우
2. 상품 또는 게시글 개수를 알 수 있는 경우
  * 1 페이지당 몇 개의 상품이나 게시글이 표시되는지 파악

In [ ]:
url = 'https://search.musinsa.com/category/001001?page={}'
data = requests.get(url.format(10))
musinsa_main = data.content
soup = BeautifulSoup(musinsa_main, 'html.parser')
soup

In [ ]:
total_page_num = int(soup.select_one('#goods_list > div.boxed-list-wrapper > div.thumbType_box.box > span.pagingNumber > span.totalPagingNum').text)
total_page_num

683

In [ ]:
from tqdm import tqdm_notebook

GOODS_URL = "https://store.musinsa.com/app/goods/{}"

url_list = []

for page in tqdm_notebook(range(1, total_page_num+1)[:1]):  # 가져올 페이지 수 [] 결정
    page_url = url.format(page)
    search_page_html = requests.get(page_url).content
    search_page_soup = BeautifulSoup(search_page_html, 'html.parser')

    products = search_page_soup.select('#searchList > li')
    print(len(products))

    for product in products:
        product_id = product['data-no'] # product id 를 뽑아올 수 있음
        product_url = GOODS_URL.format(product_id)
        url_list.append(product_url)

print(url_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


90

['https://store.musinsa.com/app/goods/1420730', 'https://store.musinsa.com/app/goods/1848166', 'https://store.musinsa.com/app/goods/996177', 'https://store.musinsa.com/app/goods/903340', 'https://store.musinsa.com/app/goods/996178', 'https://store.musinsa.com/app/goods/1886950', 'https://store.musinsa.com/app/goods/1884943', 'https://store.musinsa.com/app/goods/1900133', 'https://store.musinsa.com/app/goods/1876384', 'https://store.musinsa.com/app/goods/1845608', 'https://store.musinsa.com/app/goods/1793420', 'https://store.musinsa.com/app/goods/987149', 'https://store.musinsa.com/app/goods/1425234', 'https://store.musinsa.com/app/goods/1827193', 'https://store.musinsa.com/app/goods/1403092', 'https://store.musinsa.com/app/goods/1845485', 'https://store.musinsa.com/app/goods/1886956', 'https://store.musinsa.com/app/goods/1830175', 'https://store.musinsa.com/app/goods/996497', 'https://store.musinsa.com/app/goods/1860490', 'https://store.musinsa.com/app/goods/1356896', 'https://stor

In [ ]:
url = "http://store.musinsa.com/app/goods/1900133"

response = requests.get(url)
print(response)

<Response [403]>


Forbidden 403 으로 접속이 거부당했으므로, user_agent 를 써봄  
이 외에 다른 여러가지 해결방법이 있음

```python
user_agent = {'User-agent' : 'Mozilla/5.0'} 
# 나는 파이썬이 아니고 Mozilla 라는 브라우저야 라고 속이는 것
```

In [ ]:
url = "http://store.musinsa.com/app/goods/1900133"

user_agent = {'User-agent' : 'Mozilla/5.0'}
response = requests.get(url, headers = user_agent)
print(response)

<Response [200]>


In [ ]:
product_page = response.content
product_soup = BeautifulSoup(product_page, 'html.parser')
product_soup

## 이미지 크롤링

In [ ]:
# 이미지 썸네일 칸 데이터 가져오기
thumb_div = product_soup.select("#detail_thumb > ul > li")
len(thumb_div)

5

In [ ]:
# 이미지 썸네일 칸 데이터 가져오기
thumb_imgs = product_soup.select("#detail_thumb > ul > li > img")
len(thumb_imgs)

5

In [ ]:
thumb_imgs[0]

<img alt="thum" big_yn="Y" id="thum_0" src="//image.msscdn.net/images/goods_img/20210415/1900133/1900133_2_60.jpg" style="display: inline-block; vertical-align: middle" width="60"/>

### 문자열 변경하기

src="//image.msscdn.net/images/goods_img/20210415/1900133/1900133_2_60.jpg"  
부분만 60 을 500 으로 바꾸면 이미지 크기가 변경됨  

---

스크래이핑 할 때는 html 페이지의 **패턴**을 먼저 **파악**해야 함!!

In [ ]:
print("https:" + thumb_imgs[0]['src'])

https://image.msscdn.net/images/goods_img/20210415/1900133/1900133_2_60.jpg


In [ ]:
thumb_imgs[0]['src'].split("/")[-1].replace("_60", "_500")

'1900133_2_500.jpg'

In [ ]:
img_url_splits = thumb_imgs[0]['src'].split("/")
img_url_splits[-1] = img_url_splits[-1].replace("_60", "_500")
img_url_splits

['',
 '',
 'image.msscdn.net',
 'images',
 'goods_img',
 '20210415',
 '1900133',
 '1900133_2_500.jpg']

In [ ]:
print("https:" + '/'.join(img_url_splits))

https://image.msscdn.net/images/goods_img/20210415/1900133/1900133_2_500.jpg


In [ ]:
img_url_splits = thumb_imgs[1]['src'].split("/")
img_url_splits[-1] = img_url_splits[-1].replace("_60", "_500")
print("https:" + '/'.join(img_url_splits))

https://image.msscdn.net/images/prd_img/20210415/1900133/detail_1900133_2_500.jpg


In [ ]:
for thumb_img in thumb_imgs:
    img_url_splits = thumb_img['src'].split("/")
    img_url_splits[-1] = img_url_splits[-1].replace("_60", "_500")
    product_img_url = "https:" + '/'.join(img_url_splits)
    filename = product_img_url.split("/")[-1]

    urlretrieve(product_img_url, filename)  # urlretrieve를 이용해 썸네일 사진들을 파일로 저장

다른 상품 페이지도 잘 크롤링이 될까??

In [ ]:
# import os # 폴더를 만들기 위해 os 패키지 import

# url = "http://store.musinsa.com/app/goods/1420730"

# user_agent = {'User-agent' : 'Mozilla/5.0'}
# response = requests.get(url, headers = user_agent)

# product_page = response.content
# product_soup = BeautifulSoup(product_page, 'html.parser')

# thumb_imgs = product_soup.select("#detail_thumb > ul > li > img")

# # 폴더 만들기
# dir_name = "./{}".format(url.split("/")[-1])
# os.mkdir(dir_name)

# for thumb_img in thumb_imgs:

#     img_url_splits = thumb_img['src'].split("/")
#     img_url_splits[-1] = img_url_splits[-1].replace("_60", "_500")
#     product_img_url = "https:" + '/'.join(img_url_splits)
#     filename = product_img_url.split("/")[-1]

#     urlretrieve(product_img_url, "{}/{}".format(dir_name, filename))

FileExistsError: ignored

## 상품 정보 크롤링

브랜드, 품번 가져오기

In [ ]:
product_soup.select_one("ul.product_article")

In [ ]:
product_info_list = product_soup.select("ul.product_article > li")
print(product_info_list[0].select_one("p.product_article_contents").select_one("a").text)

In [ ]:
# 브랜드 / 상품 코드 가져옴
product_brand = product_info_list[0].select_one("p.product_article_contents > strong").text.split(" / ")[0]
product_code = product_info_list[0].select_one("p.product_article_contents > strong").text.split(" / ")[1]

print("브랜드 : {} / 코드 : {}".format(product_brand, product_code))

In [ ]:
product_season = product_info_list[1].select_one("p.product_article_contents > strong").text
print(product_season)

print 를 통해 text의 형태를 봤을 때 season과 gender를 나누기 위해서  
단순한 split이 아닌, 정규식이 필요함

In [ ]:
product_season = product_info_list[1].select_one("p.product_article_contents > strong").text
# product_season = re.sub("[ ]{2, }", "", product_season)
product_season = ' '.join(product_season.split())
print(product_season)

In [ ]:
# 시즌, 성별

import re
product_season_gender = product_info_list[1].select_one("p.product_article_contents").text.strip()
product_season_gender = re.sub("[ ]{2,}", " ", product_season_gender).split("\n/\n")

if len(product_season_gender) >1 :
    product_season = product_season_gender[0].replace("\n", "")
    product_gender = product_season_gender[1].replace("\n", "")
else:
    product_gender = product_season_gender[0].replace("\n", "")

print("시즌 : {}, 성별 : {}".format(product_season, product_gender))

In [ ]:
# 좋아요 데이터
product_info_list[4]

좋아요가 '0' 이라고 나옴  
페이지에서 새로고침을 해보면 `Network - XHR - counts - Response` 에서 좋아요 데이터를 찾을 수 있음  
`Headers`의 `Request URL`을 가져와야 함

In [ ]:
import json

user_agent = {'User-agent' : 'Mozilla/5.0'}

counts = requests.post("https://like.musinsa.com/like/api/v1/liketypes/goods/counts", data={"relationIds":["751030"]}, headers = user_agent)

data = json.loads(counts.content)
data

In [ ]:
user_agent = {'User-agent' : 'Mozilla/5.0',
              'content-type' : 'application/json'
              }

counts = requests.post("https://like.musinsa.com/like/api/v1/liketypes/goods/counts", data={"relationIds":["751030"]}, headers = user_agent)

data = json.loads(counts.content)
data

In [ ]:
# content type이 json이므로, data도 json으로 놓고 접근해야함
user_agent = {'User-agent' : 'Mozilla/5.0',
              'content-type' : 'application/json'
              }

counts = requests.post("https://like.musinsa.com/like/api/v1/liketypes/goods/counts",
                       
                       # json.dumps : 딕셔너리(dict)를 json화 시켜준다
                       data=json.dumps({"relationIds":["751030"]}),  # data가 parameter를 전달한다 (json type으로)
                       headers = user_agent)

data = json.loads(counts.content) # json을 딕셔너리(dict)화 시켜준다
data

In [ ]:
data['data']['contents'][0]['count']

In [ ]:
product_info_list[4].select_one("span.prd_like_cnt").text

In [ ]:
product_id = 751030

In [ ]:
# 사진 후기
review_photo_url = "https://store.musinsa.com/app/reviews/goods_estimate_list/photo/{}/0/{}".format(product_id, 1)
review_photo_page = requests.get(review_photo_url, headers=user_agent).content
review_photo_soup = BeautifulSoup(review_photo_page, 'html.parser')

In [ ]:
review_photo_soup.select("div.review-contents__text")[1].text.strip()

In [ ]:
# 스타일 후기
review_style_url = "https://store.musinsa.com/app/reviews/goods_estimate_list/style/{}/0/{}".format(product_id, 1)
review_style_page = requests.get(review_style_url, headers=user_agent).content
review_style_soup = BeautifulSoup(review_style_page, 'html.parser')
review_style_soup.select("div.review-contents__text")[1].text.strip()

In [ ]:
# 상품 후기
review_goods_url = "https://store.musinsa.com/app/reviews/goods_estimate_list/goods/{}/0/{}".format(product_id, 1)
review_goods_page = requests.get(review_goods_url, headers=user_agent).content
review_goods_soup = BeautifulSoup(review_goods_page, 'html.parser')
review_goods_soup.select("div.review-contents__text")[1].text.strip()

최종 버전

In [ ]:
from tqdm import tqdm_notebook

GOODS_URL = "https://store.musinsa.com/app/goods/{}"

url_list = []

for page in tqdm_notebook(range(1, total_page_num+1)[:3]):
  page_url = url.format(page)
  search_page_html = requests.get(page_url).content
  search_page_soup = BeautifulSoup(search_page_html, 'html.parser')
  
  products = search_page_soup.select("#searchList > li")
  
  for product in products:
    product_id = product['data-no']
    product_url = GOODS_URL.format(product_id)
    url_list.append(product_url)

print(url_list)

In [ ]:
import os
import re
import json

for url in url_list:
  user_agent = {'User-agent': 'Mozilla/5.0'} # 나는 파이썬이 아니고 Mozilla 라는 브라우져야! 라고 속이는 것. 이 기법이 항상 통하는 것은 아님..
  response  = requests.get(url, headers = user_agent)

  product_page = response.content
  product_soup = BeautifulSoup(product_page, 'html.parser')

  ## 이미지 크롤링 ##
  thumb_imgs = product_soup.select("#detail_thumb > ul > li > img")

  # 폴더 만들기
  dir_name = "./{}".format(url.split("/")[-1])
  os.mkdir(dir_name)

  for thumb_img in thumb_imgs:
    img_url_splits = thumb_img['src'].split("/")
    img_url_splits[-1] = img_url_splits[-1].replace("_60", "_500")
    product_img_url = "https:" + '/'.join(img_url_splits)
    filename = product_img_url.split("/")[-1]
    urlretrieve(product_img_url, "{}/{}".format(dir_name, filename))
  ###############

  # 브랜드 / 상품 코드
  product_info_list = product_soup.select("ul.product_article > li")
  product_brand = product_info_list[0].select_one("p.product_article_contents > strong").text.split(" / ")[0]
  product_code = product_info_list[0].select_one("p.product_article_contents > strong").text.split(" / ")[1]

  print("브랜드 : {} / 코드 : {}".format(product_brand, product_code))
  
  # 시즌 / 성별
  product_season_gender = product_info_list[1].select_one("p.product_article_contents").text.strip()
  product_season_gender = re.sub("[ ]{2,}", " ", product_season_gender).split("\n/\n")

  product_season = None
  product_gender = None

  if len(product_season_gender) > 1 :
    product_season = product_season_gender[0].replace("\n", "")
    product_gender = product_season_gender[1].replace("\n", "")
  else:
    product_season = product_season_gender[0].replace("\n", "")

  print("시즌 : {} / 성별 : {}".format(product_season, product_gender))


  # 좋아요 수
  user_agent = {
      'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
      "content-type": "application/json", # json으로 요청을 할거야~
  }
  counts = requests.post("https://like.musinsa.com/like/api/v1/liketypes/goods/counts",
                        
                        # json.dumps : 딕셔너리(dict)를 json화 시켜준다.
                        data=json.dumps({"relationIds":[url.split("/")[-1]]}), # data : parameter 전달 하기.. json type으로 파라미터를 전달.
                        headers = user_agent)

  data = json.loads(counts.content) # json을 딕셔너리(dict)화 시켜준다.
  print("좋아요 수 : {} ".format(data['data']['contents'][0]['count']))

  print()

FileExistsError: ignored

코랩을 이용해서 다운 받은 이미지들을 압축

`!zip -r "압축 파일 명" "압축 대상 폴더"

In [ ]:
!zip -r "/content/files.zip" "/content"

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/.last_update_check.json (deflated 24%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2021.07.01/ (stored 0%)
  adding: content/.config/logs/2021.07.01/13.40.56.775713.log (deflated 91%)
  adding: content/.config/logs/2021.07.01/13.41.17.729047.log (deflated 54%)
  adding: content/.config/logs/2021.07.01/13.41.40.038589.log (deflated 54%)
  adding: content/.config/logs/2021.07.01/13.41.33.930553.log (deflated 86%)
  adding: content/.config/logs/2021.07.01/13.41.56.945079.l

파일 다운로드

In [ ]:
from google.colab import files

files.download("files.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>